In [1]:
# Required libraries
from pathlib import Path
import sys
import os 
import pandas as pd
import numpy as np
from itertools import chain
import matplotlib.pyplot as plt
import datetime
import zipfile 
from timeit import default_timer as timer
# Paths
sys.path.append(os.path.join(Path(os.getcwd()).parent))  
data_path = os.path.join(os.path.join(Path(os.getcwd()).parent), 'data')
data_per_day_path = os.path.join(os.path.join(Path(os.getcwd()).parent), 'data','data_per_day')

In [2]:
import cupy as cp
import pandas as pd
import cudf
import dask_cudf

In [3]:
import time

from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:43291 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 62.81 GiB


In [4]:
# get the csv file now
csv_files = [f for f in os.listdir(data_per_day_path) if os.path.isfile(os.path.join(data_per_day_path, f))]
csv_files = [file for file in csv_files if '.csv' in file]
csv_files = [os.path.join(data_per_day_path, x) for x in csv_files]
csv_file = csv_files[0]
csv_file

'/home/patricklucescu/Dropbox/Projects/financial_volatility/financial_volatility/data/data_per_day/SPY_20200225.csv'

In [11]:
%%time
data_df = dask_cudf.read_csv(csv_file, npartitions=10)
data_df = data_df.persist()
data_df

CPU times: user 3.11 ms, sys: 9.18 ms, total: 12.3 ms
Wall time: 11.7 ms


,EX,BID,BIDSIZ,ASK,ASKSIZ,SYM_ROOT,SYM_SUFFIX,DT
npartitions=5,,,,,,,,
,object,float64,int64,float64,int64,object,int8,object
,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...


In [14]:
%%time
data_df.DT = data_df.DT.astype('datetime64[s]')

# non zero quotes
data_df = data_df.loc[(data_df.BID>0) & (data_df.BIDSIZ>0) & (data_df.ASK>0) & (data_df.ASKSIZ>0)]

# autoselect exchange
data_df['total_size'] = data_df.BID + data_df.ASK
data_df = data_df.loc[data_df.EX == data_df.groupby(['EX']).sum().total_size.compute().to_pandas().idxmax()]

# delete negative spreads
data_df = data_df.loc[data_df.ASK > data_df.BID]

# mergeQuotesSameTimestamp
data_df.loc[0,'EX']
#data_df = data_df.groupby(['DT']).median().compute()

CPU times: user 215 ms, sys: 0 ns, total: 215 ms
Wall time: 876 ms


<dask_cudf.Series | 400 tasks | 5 npartitions>